In [1]:
%load_ext jupyter_black
import numpy as np
from numba import cuda, float32, int32
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_normal_float32
import math
import matplotlib.pyplot as plt
import scienceplots

plt.style.use(["grid", "science", "no-latex"])

### Monte Carlo estimation using numba.cuda.reduce :

In [2]:
@cuda.reduce
def sum_reduce(a, b):
    return a + b


@cuda.jit
def asian_option_kernel_1(
    rng_states, S, I, X, X_square, n_paths, n_steps, t, dt, r, sigma
):
    """Cuda kernel that computes samples n_paths payoffs of the asian option
    Args:
        - rng_states: array of cuda random states
        - S: array of size n_paths containing the initial value of S_t
        - I: array of size n_paths containing the initial value of I_t
        - X: array of size n_paths where we will save the sampled payoffs
        - X_square: array of size n_paths where we will save the sampled payoffs squared
        - n_paths: number of sampled paths
        - n_steps: number of time steps per path
        - t: time of pricing
        - dt: square root of time to maturity / n_steps
        - r: interest rate
        - sigma: volatility
    Returns:
        - Device.array: array containing the sampled payoffs"""

    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    if tid >= n_paths:
        return  # Exit kernel to avoid index out of bounds error

    total_time = t + n_steps * dt * dt  # T = t + time to maturity
    path_integral = 0.0  # Variable where we store the path average of S
    for i in range(n_steps):
        # Generate a random normal increment
        rand = xoroshiro128p_normal_float32(rng_states, tid)
        # Increment the euler scheme
        S[tid] = S[tid] * (1.0 + r * dt * dt + sigma * dt * rand)
        # Increment the path integral
        path_integral += S[tid]

    # Update I_t with I_T
    I[tid] = dt * dt * path_integral / total_time + t * I[tid] / total_time

    # Compute payoff (S_T - I_T)_+ and it's square
    X[tid] = math.exp(-r * dt * dt * n_steps) * max(S[tid] - I[tid], 0.0)
    X_square[tid] = X[tid] * X[tid]


def asian_option_1(
    S: float,
    I: float,
    n_paths: int,
    n_steps: int,
    t: float,
    ttm: float,
    r: float,
    sigma: float,
    threadsperblock: int = 1024,
    seed: int = 42,
) -> float:
    """Monte Carlo estimation of an Asian option's price using cuda
    Args:
        - S: initial value S_t
        - I: initial value I_t
        - n_paths: number of sampled paths
        - n_steps: number of time steps per path
        - t: time of pricing
        - ttm: time to maturity
        - r: interest_rate
        - sigma: volatility"""

    blockspergrid = blockspergrid = (n_paths + (threadsperblock - 1)) // threadsperblock
    dt = np.sqrt(ttm / n_steps)

    # Initializing initial conditions on gpu memory
    dtype = np.float32
    S_gpu = cuda.to_device(S * np.ones(n_paths, dtype=dtype))
    I_gpu = cuda.to_device(I * np.ones(n_paths, dtype=dtype))
    X_gpu = cuda.to_device(np.zeros(n_paths, dtype=dtype))
    X_square_gpu = cuda.to_device(np.zeros(n_paths, dtype=dtype))

    # Initializing random states
    rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=seed)

    # Running the CUDA Kernel
    asian_option_kernel_1[blockspergrid, threadsperblock](
        rng_states, S_gpu, I_gpu, X_gpu, X_square_gpu, n_paths, n_steps, t, dt, r, sigma
    )

    mean = sum_reduce(X_gpu) / n_paths
    var = sum_reduce(X_square_gpu) / n_paths - mean**2

    return mean, var

In [3]:
n = int(1e6)
threadsperblock = 1024
blockspergrid = (n + (threadsperblock - 1)) // threadsperblock
N = 100
S = np.ones(n, dtype=np.float32) * 50
I = np.ones(n, dtype=np.float32) * 60
X = np.zeros(1, dtype=np.float32)
sigma = 0.2
r = 0.1
t = 0.2
T = 1.0
dt = math.sqrt(T / N)
rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=42)

In [8]:
mean, var = asian_option_1(50, 60, n, N, t, T, r, sigma)

In [9]:
print(
    f"Fair price is : {mean} with 95% confidence interval : [{mean - 1.95*np.sqrt(var/n)},{mean + 1.95*np.sqrt(var/n)}]"
)

Fair price is : 3.16599775 with 95% confidence interval : [3.156958623857822,3.1750368761421774]


### Monte Carlo estimation without numba.cuda.reduce : 

In [10]:
n = int(1e6)
threadsperblock = 1024
blockspergrid = (n + (threadsperblock - 1)) // threadsperblock


@cuda.jit
def asian_option_kernel_2(rng_states, S, I, X, n_paths, n_steps, t, dt, r, sigma):
    """Cuda kernel that computes samples n_paths payoffs of the asian option
    Args:
        - rng_states: array of cuda random states
        - S: array of size n_paths containing the initial value of S_t
        - I: array of size n_paths containing the initial value of I_t
        - X: array of size 2 where we will save the mean and variance
        - n_paths: number of sampled paths
        - n_steps: number of time steps per path
        - t: time of pricing
        - dt: square root of time to maturity / n_steps
        - r: interest rate
        - sigma: volatility
    Returns:
        - Device.array: array containing the sampled payoffs"""

    tid = cuda.threadIdx.x
    blocksize = cuda.blockDim.x

    idx = tid + cuda.blockIdx.x * blocksize

    if idx >= n_paths:
        return  # Exit kernel to avoid index out of bounds error

    total_time = t + n_steps * dt * dt  # T = t + time to maturity
    path_integral = 0.0  # Variable where we store the path average of S
    for i in range(n_steps):
        # Generate a random normal increment
        rand = xoroshiro128p_normal_float32(rng_states, idx)
        # Increment the euler scheme
        S[idx] = S[idx] * (1.0 + r * dt * dt + sigma * dt * rand)
        # Increment the path integral
        path_integral += S[idx]

    # Update I_t with I_T
    I[idx] = dt * dt * path_integral / total_time + t * I[idx] / total_time

    # Initializing shared memory
    xshared = cuda.shared.array(shape=(2, threadsperblock), dtype=np.float32) 
    xshared[0, tid] = (
        math.exp(-r * dt * dt * n_steps) * max(S[idx] - I[idx], 0.0) / n_paths
    )
    xshared[1, tid] = xshared[0, tid] ** 2 * n_paths

    cuda.syncthreads() # Hold until shared memory is fully initialized within the block
    i = blocksize // 2
    while i > 0:
        if tid < i:
            xshared[0, tid] += xshared[0, tid + i]
            xshared[1, tid] += xshared[1, tid + i]
        cuda.syncthreads() # Add 
        i //= 2

    if tid == 0:
        cuda.atomic.add(X, 0, xshared[0, 0])
        cuda.atomic.add(X, 1, xshared[1, 0])


def asian_option_2(
    S: float,
    I: float,
    n_paths: int,
    n_steps: int,
    t: float,
    ttm: float,
    r: float,
    sigma: float,
    threadsperblock: int = 1024,
    seed: int = 42,
) -> float:
    """Monte Carlo estimation of an Asian option's price using cuda
    Args:
        - S: initial value S_t
        - I: initial value I_t
        - n_paths: number of sampled paths
        - n_steps: number of time steps per path
        - t: time of pricing
        - ttm: time to maturity
        - r: interest_rate
        - sigma: volatility"""

    blockspergrid = blockspergrid = (n_paths + (threadsperblock - 1)) // threadsperblock
    dt = np.sqrt(ttm / n_steps)

    # Initializing initial conditions on gpu memory
    dtype = np.float32
    S_gpu = cuda.to_device(S * np.ones(n_paths, dtype=dtype))
    I_gpu = cuda.to_device(I * np.ones(n_paths, dtype=dtype))
    X_gpu = cuda.to_device(np.zeros(2, dtype=dtype))

    # Initializing random states
    rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=seed)

    # Running the CUDA Kernel
    asian_option_kernel_2[blockspergrid, threadsperblock](
        rng_states, S_gpu, I_gpu, X_gpu, n_paths, n_steps, t, dt, r, sigma
    )

    X = X_gpu.copy_to_host()

    return X[0], X[1] - X[0] ** 2

In [11]:
n = int(1e6)
threadsperblock = 1024
blockspergrid = (n + (threadsperblock - 1)) // threadsperblock
N = 100
S = np.ones(n, dtype=np.float32) * 50
I = np.ones(n, dtype=np.float32) * 60
X = np.zeros(2, dtype=np.float32)
sigma = 0.2
r = 0.1
t = 0.2
T = 1.0
dt = math.sqrt(T / N)
rng_states = create_xoroshiro128p_states(threadsperblock * blockspergrid, seed=42)

In [12]:
mean, var = asian_option_2(50, 60, n, N, t, T, r, sigma)
print(
    f"Fair price is : {mean} with 95% confidence interval : [{mean - 1.95*np.sqrt(var/n)},{mean + 1.95*np.sqrt(var/n)}]"
)

Fair price is : 3.167367696762085 with 95% confidence interval : [3.158327592762361,3.176407800761809]
